In [1]:
unit_blue = 780/421
unit_IR = 780/1013

blue_au = 415.1 # blue fabry-perot arbitrary unit
infrared_fp_au = 312.5 # infrared fabry-perot arbitrary unit
depumping_RF_Mhz = 83 # AOM+1 radio-frequency


blue_movement = +2
new_blue = blue_au + blue_movement
infrared_movement = - blue_movement*unit_blue/unit_IR
new_infrared = infrared_fp_au + infrared_movement
new_depumping = depumping_RF_Mhz + infrared_movement

print(new_infrared, new_depumping, new_blue)

307.687648456057 78.187648456057 417.1


In [46]:
a_aus = 2.512 # intercept fitted
b = 1.1584 # angular coefficient fitted
da = 0.0734
db = 0.126
time_fit = 1

rate = b/a_aus*1000
err = ((da/a_aus)**2+(db/b)**2)**(1/2)*(b/a_aus)*1000
print("taking as denominator the N_ions in the start: ", rate, "+-", err)

a = (a_aus + (a_aus-b*time_fit/2))/2

rate = b/a*1000
err = ((da/a)**2+(db/b)**2)**(1/2)*(b/a)*1000
print("taking as denominator the N_ions in the half: ", rate, "+-", err)


taking as denominator the N_ions in the start:  461.14649681528664 +- 51.93759056668979
taking as denominator the N_ions in the half:  521.2383009359252 +- 59.25146554456595
